## 5. Prompting with Python

First, we’ll install the libraries we need. The `huggingface_hub` package is the official client for Hugging Face’s API. The `rich` and `ipywidgets` packages are helper libraries that will improve how your outputs look in Jupyter notebooks.

A common way to install packages from inside your JupyterLab Desktop notebook is to use the `%pip command`. Hit the play button in the top toolbar after selecting the cell below.

In [ ]:
%pip install rich ipywidgets huggingface_hub

If the `%pip command` doesn’t work on your computer, try substituting the `!pip command` instead. Or you can install the packages from the command line on your computer and restart your notebook.

Now let's import them in the cell that appears below the installation output. Hit play again.

In [2]:
import os
from rich import print
from huggingface_hub import InferenceClient

With `api_key = os.getenv("HF_TOKEN")`, we're calling the free Hugging Face Inference API, using an authentication token stored in the "Secrets" of this Space. If you'd like to duplicate this Space, you'll need to create a token with your account [here](https://huggingface.co/settings/tokens).

You should continue adding new cells as you need throughout the rest of the class.

In [3]:
api_key = os.getenv("HF_TOKEN")

Let’s make our first prompt. To do that, we submit a dictionary to Hugging Face’s `chat.completions.create` method. The dictionary has a `messages` key that contains a list of dictionaries. Each dictionary in the list represents a message in the conversation. When the role is "user" it is roughly the same as asking a question to a chatbot.

We also need to pick a model from among the choices Hugging Face gives us. We’re picking Llama 3.3, the latest from Meta.

In [4]:
client = InferenceClient(
    model="meta-llama/Llama-3.3-70B-Instruct",
    token=api_key,
)

In [5]:
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of data journalism in a concise sentence"
        }
    ],
)

Our client saves the response as a variable. Print that Python object to see what it contains.

In [6]:
print(response)

ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, 
message=ChatCompletionOutputMessage(role='assistant', content='Data journalism plays a crucial role in holding 
institutions accountable and informing the public by analyzing and interpreting complex data to uncover trends, 
patterns, and insights that can lead to more informed decision-making and a deeper understanding of social 
issues.', tool_calls=None), logprobs=None)], created=1742869712, id='', model='meta-llama/Llama-3.3-70B-Instruct', 
system_fingerprint='3.0.1-sha-bb9095a', usage=ChatCompletionOutputUsage(completion_tokens=45, prompt_tokens=46, 
total_tokens=91), object='chat.completion')

You should see something like:

```
ChatCompletionOutput(
    choices=[
        ChatCompletionOutputComplete(
            finish_reason='stop',
            index=0,
            message=ChatCompletionOutputMessage(
                role='assistant',
                content='Data journalism plays a crucial role in holding those in power accountable by using data analysis and visualization to uncover insights, trends, and patterns that inform and engage the public on important issues.',
                tool_calls=None
            ),
            logprobs=None
        )
    ],
    created=1742592105,
    id='',
    model='meta-llama/Llama-3.3-70B-Instruct',
    system_fingerprint='3.2.1-native',
    usage=ChatCompletionOutputUsage(
        completion_tokens=37,
        prompt_tokens=46,
        total_tokens=83
    ),
    object='chat.completion'
)
```

There’s a lot here, but the `message` has the actual response from the LLM. Let’s just print the content from that message. Note that your response probably varies from this guide. That’s because LLMs mostly are probablistic prediction machines. Every response can be a little different.

In [7]:
print(response.choices[0].message.content)

Data journalism plays a crucial role in holding institutions accountable and informing the public by analyzing and 
interpreting complex data to uncover trends, patterns, and insights that can lead to more informed decision-making 
and a deeper understanding of social issues.

Let’s pick a different model from amongthe choices that Hugging Face offers. One we could try is Gemma2, an open model from Google. Rather than add a new cell, lets revise the code we already have and rerun it.

In [8]:
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of data journalism in a concise sentence",
        }
    ],
    model="google/gemma-2-9b-it", # NEW
)

Again, your response might vary from what’s here. Let’s find out.

In [9]:
print(response.choices[0].message.content)

Data journalism plays a crucial role in holding those in power accountable by uncovering hidden trends, patterns, 
and insights through the analysis and visualization of data, enabling informed decision-making and promoting 
transparency.

---
### Sidenote:
Hugging Face’s Python library is very similar to the ones offered by OpenAI, Anthropic and other LLM providers. If you prefer to use those tools, the techniques you learn here should be easily transferable.

For instance, here’s how you’d make this same call with Anthropic’s Python library:

In [ ]:
from anthropic import Anthropic

client = Anthropic(api_key=api_key)

response = client.messages.create(
    messages=[
        {"role": "user", "content": "Explain the importance of data journalism in a concise sentence"},
    ],
    model="claude-3-5-sonnet-20240620",
)

print(response.content[0].text)

---
A well-structured prompt helps the LLM provide more accurate and useful responses.

One common technique for improving results is to open with a “system” prompt to establish the model’s tone and role. Let’s switch back to Llama 3.3 and provide a `system` message that provides a specific motivation for the LLM’s responses.

In [11]:
response = client.chat.completions.create(
    messages=[
        ### <-- NEW
        {
            "role": "system",
            "content": "you are an enthusiastic nerd who believes data journalism is the future."
        },
        ### -->
        {
            "role": "user",
            "content": "Explain the importance of data journalism in a concise sentence",
        }
    ],
    model="meta-llama/Llama-3.3-70B-Instruct", # NEW
)

Check out the results.

In [12]:
print(response.choices[0].message.content)

Data journalism is revolutionizing the way we consume news by using statistical analysis and visualizations to 
uncover hidden truths, provide fact-based evidence, and hold those in power accountable, thereby fostering a more 
informed and engaged citizenry!

Want to see how tone affects the response? Change the system prompt to something old-school.

In [13]:
response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "you are a crusty, ill-tempered editor who hates math and thinks data journalism is a waste of time and resources." # NEW
        },
        {
            "role": "user",
            "content": "Explain the importance of data journalism in a concise sentence",
        }
    ],
    model="meta-llama/Llama-3.3-70B-Instruct",
)

Then re-run the code and summon J. Jonah Jameson.

In [14]:
print(response.choices[0].message.content)

If I must, data journalism is supposedly important because it allows reporters to use numbers and statistics to 
fact-check claims, identify trends, and hold those in power accountable, but frankly, I've seen better storytelling
in a spreadsheet.

**[6. Structured responses →](ch6-structured-responses.ipynb)**